In [ ]:
import numpy as np
import math
import pandas as pd
import os
import mne
import matplotlib.pyplot as plt
from scipy.signal import welch, hamming
from fooof import FOOOF

In [ ]:
folder_paths = ['data\\decreased_cognition', 'data\\intact_cognition']

set_files = []
for folder_path in folder_paths:
    files = []
    for f in os.listdir(folder_path):
        if f.endswith('.set'):
            files.append(os.path.join(folder_path, f))
    set_files.extend(files)
print(set_files)

In [ ]:
data = []
for file_path in set_files:
    if 'data\\decreased_cognition' in file_path:
        label = 0
    else:
        label = 1
    data.append([file_path, label])
df = pd.DataFrame(data, columns=['file_path', 'label'])
df.to_csv('Y.csv', index=False)

In [ ]:
def process_file(file_path):
    raw = mne.io.read_raw_eeglab(file_path, preload=True)
    channel_name = raw.info['ch_names']
    channel_index = []
    channel_data = [] 
    channel_time = []
    frequencies = []
    psd = []
    all_features = []
    for i in range(19):
        index = raw.ch_names.index(channel_name[i])
        data = raw[index, :][0] * 1e6
        time = raw[index, :][1]

        channel_index.append(index)
        channel_data.append(data)
        channel_time.append(time)
        freqs, power = welch(channel_data[i], fs=raw.info['sfreq'], nperseg=512, window=hamming(512))
        frequencies.append(freqs)
        psd.append(power)
        
        fm = FOOOF(peak_width_limits=[0.05, 20], max_n_peaks=1, min_peak_height=0.01, 
                   peak_threshold=-5, aperiodic_mode='fixed')
        freq_range = [4, 16]
        fm.report(frequencies[i], psd[i][0], freq_range)
        
        aperiodic_params = fm.get_params('aperiodic_params')
        peak_params = fm.get_params('peak_params')
        peak_params = peak_params.flatten()
        
        if np.isnan(peak_params).any():
            peak_params = np.array([0, 0, 0])
        features = np.concatenate((peak_params, aperiodic_params))
        all_features.extend(features)
    return all_features 

In [ ]:
columns = []
for i in range(19):
    columns.extend([
        f'ch{i+1}_aperiodic_offset',
        f'ch{i+1}_aperiodic_exponent',
        f'ch{i+1}_peak_cf',
        f'ch{i+1}_peak_pw',
        f'ch{i+1}_peak_bw'
    ])
df = pd.DataFrame(columns=columns)
for set_file in set_files:
    features = process_file(set_file)
    if len(features) == 95:
        # Thêm các đặc trưng vào DataFrame
        df = df.append(pd.Series(features, index=columns), ignore_index=True)
df.to_csv('X.csv', index=False)
print("done")